<a href="https://colab.research.google.com/github/sumukhbhat12/Natural-Language-Processing-Course/blob/main/NLTP_Course_simple_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The dataset used is IMDB 50k movie reviews dataset available in kaggle

https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [1]:
import pandas as pd
import numpy as np

**import the csv dataset using pd.read_csv() method**

In [13]:
dataset = pd.read_csv('/content/IMDB Dataset.csv',on_bad_lines=None)

print(dataset.shape)

print(dataset.head(10))

(50000, 2)
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
5  Probably my all-time favorite movie, a story o...  positive
6  I sure would like to see a resurrection of a u...  positive
7  This show was an amazing, fresh & innovative i...  negative
8  Encouraged by the positive comments about this...  negative
9  If you like original gut wrenching laughter yo...  positive


check for null values

In [15]:
print(dataset.isnull().sum())
print(dataset['sentiment'].value_counts())

review       0
sentiment    0
dtype: int64
positive    25000
negative    25000
Name: sentiment, dtype: int64


**Text Normalization**

imports

In [17]:
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize

import spacy
import re, string, unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob, Word
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from bs4 import BeautifulSoup

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [21]:
#Tokenization of text
tokenizers = ToktokTokenizer()

#English Stopwords
stopwords = nltk.corpus.stopwords.words('english')

# print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Remove the noise from the text

In [22]:
def noiseremoval_text(text):
  #BeautifulSoup is used to remove the html or xml tags from the data
  soup = BeautifulSoup(text, "html.parser")
  text = soup.get_text()
  text = re.sub('\[[^]]*\]', '', text)
  return text

In [36]:
dataset['review'] = dataset['review'].apply(noiseremoval_text)

<ipython-input-22-9de83bd13445>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


**Stemming**

In [28]:
def stemmer(text):
  ps = PorterStemmer()
  text = ' '.join([ps.stem(word) for word in text.split()])
  return text

In [30]:
dataset.head(10)

,review,sentiment
0,one of the other review ha mention that after ...,positive
1,a wonder littl production. the film techniqu i...,positive
2,i thought thi wa a wonder way to spend time on...,positive
3,basic there' a famili where a littl boy (jake)...,negative
4,"petter mattei' ""love in the time of money"" is ...",positive
5,"probabl my all-tim favorit movie, a stori of s...",positive
6,i sure would like to see a resurrect of a up d...,positive
7,"thi show wa an amazing, fresh & innov idea in ...",negative
8,encourag by the posit comment about thi film o...,negative
9,if you like origin gut wrench laughter you wil...,positive


**Remove Stopwords**

In [31]:
def removing_stopwords(text, is_lower_case=False):
  tokenizers = ToktokTokenizer()
  tokens = tokenizers.tokenize(text)
  tokens = [i.strip() for i in tokens]
  filtokens = [i for i in tokens if i.lower() not in stopwords]
  filtered_text = ' '.join(filtokens)
  return filtered_text

In [37]:
dataset['review'] = dataset['review'].apply(removing_stopwords)

In [ ]:
dataset['review'] = dataset['review'].apply(stemmer)

In [38]:
dataset.head(10)

,review,sentiment
0,one review ha mention watch 1 oz episod ' hook...,positive
1,wonder littl production. film techniqu veri un...,positive
2,thought thi wa wonder way spend time hot summe...,positive
3,basic ' famili littl boy ( jake ) think ' zomb...,negative
4,"petter mattei ' "" love time money "" visual stu...",positive
5,"probabl all-tim favorit movie , stori selfless...",positive
6,sure would like see resurrect date seahunt ser...,positive
7,"thi show wa amazing , fresh &amp; ; innov idea...",negative
8,encourag posit comment thi film wa look forwar...,negative
9,like origin gut wrench laughter like thi movie...,positive
